In [1]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

## The panelists by country

Reading in the panel information Netquest gave us.

In [2]:
panels = pd.read_csv("../Data/All countries export/20190814/VANDUS 1147438_W3_20190814.csv",
                     sep=';', 
#                      skiprows=2891450,
#                      nrows=1000000,
                     na_values=[' ','.'], 
                     encoding='utf8',
                     #error_bad_lines=False
                    )

C:\Users\schadem\AppData\Local\Continuum\anaconda3\envs\idb\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (535,536,537) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
panels['AR_localidad'].unique()

array([  nan,  463.,  397., ..., 1597.,  774.,   24.])

Creating a dictionary of individual country dataframes, and cleaning them up

In [5]:
panels_dict = {}
for p in panels.pais.unique():
    panels_dict[p] = panels[panels.pais==p]#.dropna(axis=1,how='all')

In [6]:
panels_dict['AR']

,p_codigo,p_sexo,panelistAge,pv_fecha_modificacion,pais,CodigoPostal,AR_provincia,AR_localidad,BR_estado,BR_cidade,...,P8,P10,P11,P12,P13,CLCSOCIAL2012,CLCSOCIAL2015,CLCSOCIAL2018,MXCSOCIAL2018,AR_departamento
19,00005191214f30f2,1,39,2017/07,AR,5411,14.0,2674.0,NaN,NaN,...,5.0,NaN,2.0,1.0,3.0,NaN,NaN,NaN,NaN,379.0
20,00005234bd11c7f7,1,40,2019/02,AR,7400,2.0,397.0,NaN,NaN,...,6.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,91.0
24,00005782b0268430,2,21,2017/12,AR,5600,5.0,2162.0,NaN,NaN,...,4.0,NaN,1.0,3.0,3.0,NaN,NaN,NaN,NaN,194.0
33,000070f97c30847d,1,36,2018/12,AR,5800,3.0,1121.0,NaN,NaN,...,4.0,NaN,2.0,3.0,3.0,NaN,NaN,NaN,NaN,148.0
42,0000884c11d14552,2,23,2017/11,AR,5300,21.0,1989.0,NaN,NaN,...,2.0,NaN,3.0,2.0,2.0,NaN,NaN,NaN,NaN,463.0
59,0000dbb91fc92ba3,1,27,2017/04,AR,2000,4.0,3094.0,NaN,NaN,...,2.0,NaN,3.0,3.0,1.0,NaN,NaN,NaN,NaN,171.0
63,0000fdf4ca7aa439,1,55,2018/09,AR,NaN,3.0,786.0,NaN,NaN,...,99.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,154.0
68,00011d73539f8740,1,21,2018/03,AR,NaN,2.0,307.0,NaN,NaN,...,2.0,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,69.0
73,000128b186b01b4d,2,34,2018/03,AR,8300,17.0,2327.0,NaN,NaN,...,4.0,NaN,2.0,2.0,3.0,NaN,NaN,NaN,NaN,411.0
82,00014bc1839d8f50,2,19,2019/04,AR,1888,2.0,206.0,NaN,NaN,...,5.0,NaN,1.0,3.0,3.0,NaN,NaN,NaN,NaN,42.0


In [5]:
for pais, data in panels_dict.items():
    if not os.path.isdir("../Data/"+pais):
        os.mkdir("../Data/"+pais)
    data.to_csv("../Data/"+pais+"/"+pais+"_netquest-panel.csv", encoding='utf8')

Next, bring the variables and labels in order.

In [7]:
variables = pd.read_excel("../Data/All countries export/manual_variables.xlsx")
levels = pd.read_excel("../Data/All countries export/manual_levels.xlsx")

In [8]:
variables.Variable.apply(
    lambda s: sum([k in s for k in other_countries])
)==0

NameError: name 'other_countries' is not defined

In [ ]:
other_countries

In [ ]:
variables_dict = {}
levels_dict = {}

for pais in panels_dict:
    other_countries = [p for p in panels_dict if not p==pais]
    var_df = variables[(variables.Variable.str.startswith(pais)) #country-specific var
                       | (variables.Variable.apply(lambda s: sum([k in s for k in other_countries]))==0)] #not specific for another country
    lev_df = levels[levels.Variable.isin(var_df.Variable)]
    
    variables_dict[pais] = var_df
    levels_dict[pais] = lev_df
    
    var_df.to_excel('../Data/'+pais+'/'+pais+'_variables.xlsx')
    lev_df.to_excel('../Data/'+pais+'/'+pais+'_levels.xlsx')

In [ ]:
MX = pd.read_csv('../Data/MX/MX_netquest-panel.csv')

In [ ]:
print(sorted(MX.MX_municipio_name1.unique().astype(str)))